# **Proyek Recommendations System: [Anime Recommendations Database]**

- **Nama:** Satria Dirgantara Nurayaman
- **Email:** satriadirgantaranuryaman@gmail.com
- **ID Dicoding:** Satria Dirgantara Nuryaman

## **Import Library**

In [1]:
# Manipulation Data
import pandas as pd
import numpy as np
import re

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Reccommender System
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# TensorFlow
import tensorflow as tf

# Configurations
import warnings 
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')

## **Data Loading**

In [2]:
# Load Anime.csv file
anime_df = pd.read_csv('Data/anime.csv')
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [3]:
# Load Rating.csv file
rating_df = pd.read_csv('Data/rating.csv')
rating_df.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


## **Exploratory Data Analysis (EDA)**

### **Deskripsi Variable**

**Anime.csv:**
| Kolom      | Deskripsi                                                                 |
|------------|--------------------------------------------------------------------------|
| anime_id   | ID unik untuk setiap anime (mengacu pada myanimelist.net)                |
| name       | Nama lengkap anime                                                       |
| genre      | Daftar genre anime, dipisahkan koma                                      |
| type       | Tipe anime (TV, Movie, OVA, dll)                                         |
| episodes   | Jumlah episode (1 jika movie)                                            |
| rating     | Rata-rata rating dari seluruh user (skala 1-10)                          |
| members    | Jumlah anggota komunitas yang memasukkan anime ini ke dalam grup mereka  |

**Rating.csv**
| Kolom    | Deskripsi                                                                 |
|----------|--------------------------------------------------------------------------|
| user_id  | ID unik pengguna (acak, tidak dapat diidentifikasi)                      |
| anime_id | ID anime yang dirating oleh user (mengacu pada anime_id di anime.csv)    |
| rating   | Nilai rating yang diberikan user (skala 1-10, -1 jika tidak memberi rating)|

### **Informasi Umum**

In [ ]:
# Informasi singkat tentang dataset anime
print(f'Jumlah Baris dan Kolom anime_df adalah: {anime_df.shape}')
print(f'\n Informasi Tentang Dataset Anime :')

# Menampilkan informasi dasar
anime_df.info()

Jumlah Baris dan Kolom anime_df adalah: (12294, 7)

 Informasi Tentang Dataset Anime :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


Berdasarkan hasil eksplorasi awal: 
- Dataset anime terdiri dari **12.294** baris dan **7** fitur utama. 
- Dataset ini juga terdiri dari **3** fitur bertipe data numerik seperti `anime_id`, `rating`, dan juga `members`, dan **4** bertipe data object seperti `name`, `genre`, `type`, dan juga `episodes`.
- Sebagian besar data sudah lengkap, namun terdapat beberapa nilai kosong pada kolom `genre`, `type`, dan `rating`. Hal ini perlu diperhatikan pada tahap data preparation agar tidak mempengaruhi hasil rekomendasi. 

Secara umum, dataset ini sudah sangat representatif untuk membangun sistem rekomendasi berbasis konten dan kolaboratif.

Pada Informasi Umum diatas ditemukan bahwa:
- Data Anime.csv memiliki 12294 baris, dan 7 fitur

In [ ]:
# Informasi singkat tentang dataset rating
print(f'Jumlah Baris dan Kolom rating_df adalah: {rating_df.shape}')
print(f'\nInformasi Tentang Dataset Rating :')

# Menampilkan informasi dasar
rating_df.info()

Jumlah Baris dan Kolom rating_df adalah: (7813737, 3)

Informasi Tentang Dataset Rating :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813737 entries, 0 to 7813736
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 178.8 MB


Berdasarkan hasil eksplorasi awal ditemukan bahwa:
- Dataset rating terdiri dari **7.813.737** baris dan **3** fitur utama, yaitu user_id, anime_id, dan rating. Seluruh data pada rating.csv bertipe numerik dan tidak ditemukan nilai kosong. 
- Tapi, jumlah data yang sangat besar ini dapat menimbulkan tantangan komputasi, terutama pada proses training model collaborative filtering. 

Jika nanti proses komputasi sangat lambat, maka rencananya kita bakal mengambil sebanyak **1 - 2 juta sampel saja**.

### **Ringkasan Statistik**

In [ ]:
# Ringkasan statistik deskriptif untuk anmie_df
print('\nStatistik Deskriptif untuk anime_df:')

# Fitur Numerik
print('Fitur Numerik:')
display(anime_df.describe())

# Fitur Kategorikal
print('Fitur Kategorikal:')
display(anime_df.describe(include='object'))

Statistik Deskriptif untuk anime_df:
Fitur Numerik:


,anime_id,rating,members
count,12294.000000,12064.000000,1.229400e+04
mean,14058.221653,6.473902,1.807134e+04
std,11455.294701,1.026746,5.482068e+04
min,1.000000,1.670000,5.000000e+00
25%,3484.250000,5.880000,2.250000e+02
50%,10260.500000,6.570000,1.550000e+03
75%,24794.500000,7.180000,9.437000e+03
max,34527.000000,10.000000,1.013917e+06


Fitur Kategorikal:


,name,genre,type,episodes
count,12294,12232,12269,12294
unique,12292,3264,6,187
top,Shi Wan Ge Leng Xiaohua,Hentai,TV,1
freq,2,823,3787,5677


Dari ringkasan statistik dataset `anime_df`, terdapat beberapa hal menarik yang dapat dicatat:

- Untuk fitur numerik:
  - **Rating** anime memiliki rata rata **6.47** dari skala **10**, dengan nilai minimum **1.67** dan maksimum **10.0**, menunjukkan adanya sebaran kualitas dari yang sangat rendah hingga sangat tinggi.
  - **Members**, yang merepresentasikan popularitas, sangat bervariasi, dengan median sebesar **1.550** ribu, namun ada anime yang mencapai lebih dari **1 juta anggota**, menunjukkan distribusi yang sangat tidak merata (right-skewed).
  
- Untuk fitur kategorikal:
  - Terdapat **12.292 judul anime unik** dari **12.294 data**, menunjukkan nyaris tidak ada duplikasi nama.
  - Genre paling sering muncul adalah **Hentai**, dengan **823 kemunculan**, sedangkan jenis penayangan paling dominan adalah **TV** (3787 data).
  - Fitur `episodes` memiliki **187 nilai unik**, dengan episode `1` paling umum muncul (5677 kali), mengindikasikan bahwa pengguna sering menonton anime Movie atau mungkin OVA dan Special Episode.

Secara keseluruhan, dataset ini merepresentasikan dunia anime dengan sangat beragam, baik dari segi rating, popularitas, genre, hingga format tayangan.

In [27]:
# Ringkasan statistik dari dataset rating_df
print('\nStatistik Deskriptif untuk rating_df:')
rating_df.describe()


Statistik Deskriptif untuk rating_df:


,user_id,anime_id,rating
count,7.813737e+06,7.813737e+06,7.813737e+06
mean,3.672796e+04,8.909072e+03,6.144030e+00
std,2.099795e+04,8.883950e+03,3.727800e+00
min,1.000000e+00,1.000000e+00,-1.000000e+00
25%,1.897400e+04,1.240000e+03,6.000000e+00
50%,3.679100e+04,6.213000e+03,7.000000e+00
75%,5.475700e+04,1.409300e+04,9.000000e+00
max,7.351600e+04,3.451900e+04,1.000000e+01


Dari ringkasan statistik deskriptif rating_df kita menemukan:

- Dataset ini mencatat lebih dari **7,8 juta interaksi** antara pengguna dan anime. Rata rata rating berada di angka **6.14**, dengan median **7.0**, menandakan kecenderungan pengguna atau penonton memberi nilai cukup positif. 

- Nilai rating berkisar dari **-1 hingga 10**, di mana **-1** kemungkinan besar berarti belum memberikan penilaian. Penyebaran rating cukup lebar, dengan standar deviasi **3.72**, menunjukkan adanya variasi opini yang tinggi. 

Secara keseluruhan, data ini menggambarkan sistem rating yang aktif dan sangat bervariasi, pondasi yang sangat kaya untuk membangun sistem rekomendasi.

### **Identifikasi Missing Values dan Duplikasi**

In [48]:
# Identifikasi Missing Values dan Duplikasi anime_df
missing_values_anime = anime_df.isnull().sum()
missing_values_anime_percentage = (missing_values_anime / len(anime_df)) * 100
duplicates_anime = anime_df.duplicated().sum()

# Membuat DataFrame untuk missing_anime_df
missing_anime_df = pd.DataFrame({
    'Jumlah Missing Values': missing_values_anime,
    'Persentasi Missing Values': missing_values_anime_percentage,
    'Jumlah Duplikasi': duplicates_anime
}).sort_values(by='Jumlah Missing Values', ascending=False)

# Menampilkan DataFrame missing_anime_df
display(missing_anime_df)

,Jumlah Missing Values,Persentasi Missing Values,Jumlah Duplikasi
rating,230,1.870831,0
genre,62,0.504311,0
type,25,0.203351,0
anime_id,0,0.000000,0
name,0,0.000000,0
episodes,0,0.000000,0
members,0,0.000000,0


- Secara keseluruhan, dataset `anime_df` tergolong bersih. Hanya terdapat sedikit missing values pada kolom `rating` (1.87%), `genre` (0.50%), dan `type` (0.20%). Sisanya lengkap 100%.
- Tidak ditemukan **data duplikat** di seluruh kolom, termasuk `anime_id` dan `name`, yang berarti tiap entri mewakili satu anime yang unik.



In [49]:
# Identifikasi Missing Values dan Duplikasi rating_df
missing_values_anime = rating_df.isnull().sum()
missing_values_anime_percentage = (missing_values_anime / len(rating_df)) * 100
duplicates_anime = rating_df.duplicated().sum()

# Membuat DataFrame untuk missing_rating_df
missing_rating_df = pd.DataFrame({
    'Jumlah Missing Values': missing_values_anime,
    'Persentasi Missing Values': missing_values_anime_percentage,
    'Jumlah Duplikasi': duplicates_anime
}).sort_values(by='Jumlah Missing Values', ascending=False)

# Menampilkan DataFrame missing_rating_df
display(missing_rating_df)

,Jumlah Missing Values,Persentasi Missing Values,Jumlah Duplikasi
user_id,0,0.0,1
anime_id,0,0.0,1
rating,0,0.0,1


- Dataset rating_df sangat bersih, tidak ada missing values yang teridentifikasi, tapi hanya aja duplikasi data saja dengan jumlah hanya 1.